In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`.

In [ ]:
def counter_file(path):
  counter = {}
  with open(path,'r', encoding='Windows-1251') as file:
      for i in file.read():
        try:
          counter[i.lower()] += 1
        except:
          counter[i.lower()] = 0
  print(counter)

igrok_counter = counter_file('/content/drive/MyDrive/10_parallel/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt')
pin_counter = counter_file('/content/drive/MyDrive/10_parallel/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt')

{'с': 11506, 'п': 5488, 'а': 18235, 'и': 13586, 'б': 3979, 'о': 23129, ',': 6371, ' ': 45075, 'ч': 4112, 'т': 14244, 'к': 6743, 'л': 9960, 'н': 14239, 'г': 3947, 'у': 6043, 'в': 9397, 'е': 20053, 'й': 2027, 'э': 835, 'р': 9481, 'b': 219, 'o': 376, 'k': 20, 's': 428, 'c': 323, 'a': 589, 'f': 51, 'e': 1199, '.': 2953, 'n': 458, 't': 331, ':': 211, 'h': 226, 'p': 99, '/': 19, '\n': 2733, 'u': 284, 'r': 307, 'd': 191, 'v': 86, 'i': 368, 'y': 7, '_': 3, '-': 899, '1': 45, '0': 21, '9': 35, '6': 41, 'm': 400, 'l': 570, 'ж': 2296, 'д': 6680, 'х': 1534, 'ф': 633, 'м': 7105, 'g': 72, '2': 41, '4': 41, '7': 41, 'я': 5457, '!': 717, '(': 275, 'з': 3354, ')': 275, 'ц': 816, 'ь': 4856, 'ш': 1942, 'ы': 3868, 'ю': 1322, ';': 405, 'щ': 586, '?': 570, 'ъ': 62, '—': 1725, '\xa0': 1471, '…': 279, '«': 128, '[': 96, ']': 96, '»': 127, "'": 58, '3': 39, '5': 39, 'q': 49, 'j': 39, '8': 37, 'w': 6, 'x': 23, 'z': 43, '`': 8}
{'с': 50083, 'п': 25651, 'а': 73554, 'и': 62029, 'б': 16015, 'о': 106739, ',': 26972,

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс.

In [ ]:
from multiprocessing import Process, Pool

p1 = Process(target = counter_file, args = ('/content/drive/MyDrive/10_parallel/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt',))
p2 = Process(target = counter_file, args = ('/content/drive/MyDrive/10_parallel/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt',))
p1.start()
p2.start()


## Лабораторная работа 10

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [ ]:
import csv
def razb_na_csv(name,data):
  with open(f'{name}.csv','w',newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['name','id','minutes','contributor_id','submitted','tags','n_steps','steps','description','ingredients','n_ingredients'])
    writer.writerows(data)

In [ ]:
with open('/content/drive/MyDrive/recipes_full.csv',newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    data = list(spamreader)
    first_slice = 1
    second_slice = 287500
    step = 287500
    procs = []
    for i in range(8):
      if i != 8:
        data_in_func = data[first_slice:second_slice]
        first_slice += step
        second_slice += step
      else:
        data_in_func = data[first_slice:]
      p = Process(target = razb_na_csv, args = (f'razb_csv_{i}',data_in_func))
      procs.append(p)
    [p.start() for p in procs]

In [ ]:
with open('/content/razb_csv_0.csv',newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    data = list(spamreader)
    print(data[0])
    print(data[1])

['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags', 'n_steps', 'steps', 'description', 'ingredients', 'n_ingredients']
['vant ivoire mickies nothing', '683970', '33', '803776', '2019-08-22', "['mexican', 'healthy-2', 'orange-roughy', 'chicken-thighs-legs', 'freezer', 'whitefish', 'pork-sausage']", '4', "['remove the fat cap and any large areas or pockets of external fat that can be easily trimmed away', 'cut the eggplants in half lengthwise and , taking care not to break the skins , gently scoop out the flesh', 'bake uncovered in a 350f for one hour or until golden and hot through', 'refrigerate']", "pat and gina neely and their family own and operate some of tennessee's best - and devilishly delicious - barbecue restaurants. but when they are relaxing at home in memphis, they love to create tasty dishes their whole family can enjoy. now they're ready to share secrets from their famous restaurant dishes.  here is the recipe for their barbecue chicken.  courtesy of food net

In [ ]:
with open('/content/razb_csv_7.csv',newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    data = list(spamreader)
    print(data[0])
    print(data[1])

['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags', 'n_steps', 'steps', 'description', 'ingredients', 'n_ingredients']
['azumaya', '563556', '51', '86764', '2005-09-08', "['picnic', 'polynesian', 'egyptian', 'lemon']", '2', "['season chops well with the salt and pepper , spread on the mustard or a paste of horseradish and vinegar', 'season with salt and pepper']", 'eating this cake is like having a chocolate bar in cake form. melts in your mouth and explodes with a full, rich, chocolate flavor. an unusual mixing method assures success every time. frost with 1.5 - 2 recipes of my semi-classic butter cream. we like white chocolate flavor, but milk chocolate works well too. fill the center with raspberry jam for a special surprise. adapted from rose beranbaum\'s "the cake bible".', "['pastry for single-crust pie', '8-grain bread', 'sourdough bread rounds', 'tofu crumbles', 'roasted garlic']", '5']


2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [ ]:
import pandas as pd

def count_avg_of_tags(file_name):
  df = pd.read_csv(file_name,usecols =  ['tags', 'n_steps'])
  df['tags'] = df['tags'].apply(lambda x: x.strip('][').split(', '))
  df = df.explode('tags')
  df = df.groupby('tags').mean()
  return dict(df)


In [ ]:
df = count_avg_of_tags('/content/razb_csv_1.csv')
print(df)

{'n_steps': tags
''                       3.480844
'1-day-or-more'          4.430428
'15-minutes-or-less'     4.950319
'3-steps-or-less'        4.698720
'30-minutes-or-less'     7.596583
                           ...   
'wings'                  3.725061
'winter'                 5.618585
'yams-sweet-potatoes'    4.161735
'yeast'                  5.316357
'zucchini'               3.533935
Name: n_steps, Length: 552, dtype: float64}


3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы иметь возможность получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.

In [ ]:
import time

def join_dicts(list_of_dict):
  final_dict = {}
  first_dict = list(list_of_dict[0].keys())
  for i in range(len(first_dict)):
    key = first_dict[i]
    value = sum([d[key] for d in list_of_dict]) / 8
    final_dict[key] = value
  return final_dict

def new_count_avg_of_tags(list_of_files):
  list_of_dicts = []
  for file_name in list_of_files:
    df = pd.read_csv(file_name,usecols =  ['tags', 'n_steps'])
    df['tags'] = df['tags'].apply(lambda x: x.strip('][').split(', '))
    df = df.explode('tags')
    df = df.groupby('tags').mean()
    list_of_dicts.append(dict(df))
  final_dict = join_dicts(list_of_dicts)
  return final_dict


In [ ]:
start_time = time.time()
list_file =  ['/content/razb_csv_0.csv','/content/razb_csv_1.csv', '/content/razb_csv_2.csv', '/content/razb_csv_3.csv', '/content/razb_csv_4.csv', '/content/razb_csv_5.csv', '/content/razb_csv_6.csv', '/content/razb_csv_7.csv']
d = new_count_avg_of_tags(list_file)
print(f'Время выполнения = {time.time() - start_time}s')
print(d)


Время выполнения = 28.706233501434326s
{'n_steps': tags
''                       3.520564
'1-day-or-more'          4.457804
'15-minutes-or-less'     4.986537
'3-steps-or-less'        4.729245
'30-minutes-or-less'     7.608965
                           ...   
'wings'                  3.699909
'winter'                 5.499454
'yams-sweet-potatoes'    4.007175
'yeast'                  5.178845
'zucchini'               3.587606
Name: n_steps, Length: 552, dtype: float64}


4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [ ]:
start_time = time.time()
pool = Pool(processes = 8)
list_of_dicts = pool.map(count_avg_of_tags, list_file)
final_dict = join_dicts(list_of_dicts)
print(final_dict)
print(f'Время выполнения = {time.time() - start_time}s')

{'n_steps': tags
''                       3.520564
'1-day-or-more'          4.457804
'15-minutes-or-less'     4.986537
'3-steps-or-less'        4.729245
'30-minutes-or-less'     7.608965
                           ...   
'wings'                  3.699909
'winter'                 5.499454
'yams-sweet-potatoes'    4.007175
'yeast'                  5.178845
'zucchini'               3.587606
Name: n_steps, Length: 552, dtype: float64}
Время выполнения = 29.845372438430786s


5. (*) Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы.

In [ ]:
import multiprocessing
import time, random

def count_avg_of_tags(file_name):
  df = pd.read_csv(file_name,usecols =  ['tags', 'n_steps'])
  df['tags'] = df['tags'].apply(lambda x: x.strip('][').split(', '))
  df = df.explode('tags')
  df = df.groupby('tags').mean()
  return dict(df)

In [ ]:
def worker(input, output):
    for func, args in iter(input.get, 'STOP'):
        result = calculate(func, args)
        output.put(result)

def calculate(func, args):
    proc_name = multiprocessing.current_process().name
    result = func(*args)
    return f'Результат функции {result}'

def plus(a, b):
    time.sleep(0.5*random.random())
    return a + b

def test():
    NUMBER_OF_PROCESSES = 3
    TASKS1 = [(count_avg_of_tags, (i, )) for i in list_file]

    task_queue = multiprocessing.Queue()
    done_queue = multiprocessing.Queue()

    for task in TASKS1:
        task_queue.put(task)

    for i in range(NUMBER_OF_PROCESSES):
        multiprocessing.Process(target=worker, args=(task_queue, done_queue)).start()

    print('НЕУПОРЯДОЧЕННЫЕ РЕЗУЛЬТАТЫ:\n')
    print('TASKS1:\n')
    for i in range(len(TASKS1)):
        print('\t', done_queue.get())

    print('STOP.')
    for i in range(NUMBER_OF_PROCESSES):
        task_queue.put('STOP')

if __name__ == '__main__':
    start_time = time.time()
    test()
    print(f'Время выполнения = {time.time() - start_time}s')

НЕУПОРЯДОЧЕННЫЕ РЕЗУЛЬТАТЫ:

TASKS1:

	 Результат функции {'n_steps': tags
''                       3.480844
'1-day-or-more'          4.430428
'15-minutes-or-less'     4.950319
'3-steps-or-less'        4.698720
'30-minutes-or-less'     7.596583
                           ...   
'wings'                  3.725061
'winter'                 5.618585
'yams-sweet-potatoes'    4.161735
'yeast'                  5.316357
'zucchini'               3.533935
Name: n_steps, Length: 552, dtype: float64}
	 Результат функции {'n_steps': tags
''                       3.520305
'1-day-or-more'          4.566628
'15-minutes-or-less'     5.038862
'3-steps-or-less'        4.719237
'30-minutes-or-less'     7.641802
                           ...   
'wings'                  3.650607
'winter'                 5.386007
'yams-sweet-potatoes'    3.933785
'yeast'                  5.044993
'zucchini'               3.644776
Name: n_steps, Length: 552, dtype: float64}
	 Результат функции {'n_steps': tags
''             